In [1]:
import numpy as np
import pandas as pd
import random
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as k
import os

/home/nextg3/miniconda3/envs/fl/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-06 12:00:55.557416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 12:00:55.802308: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-06 12:00:55.839482: W tensorflow/compiler/xla/stream_executor/plat

In [2]:
def load(paths, verbose=-1):
    data = list()
    labels = list()
    # Loop over input images
    for (i,imgpath) in enumerate(paths):
        #  load image and extract label
        im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        image = np.array(im_gray).flatten()
        label = imgpath.split(os.path.sep)[-2]
        #  scale image
        data.append(image/255)
        labels.append(label)
        #  show update
        if verbose > 0 and i > 0 and (i+1) % verbose ==  0:
            print("[INFO] processed {}/{}".format(i + 1, len(paths)))
        #  return tuple mof data and label
    return data,labels

In [3]:
#  declare path
# img_path = '/home/nextg3/Documents/FederatedLearning/FL_final/MNIST_training'
img_path = '/home/nextg3/Documents/Thesis/Code/FL/MNIST_training'
# Get path list
image_paths = list(paths.list_images(img_path))
#  apply function
image_list, label_list = load(image_paths, verbose = 10000)
#  Binarize the labels
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)
# split train test
X_train,X_test, y_train, y_test = train_test_split(image_list,
                                                   label_list,
                                                   test_size=0.1,
                                                   random_state=37)

[INFO] processed 10000/60000
[INFO] processed 20000/60000
[INFO] processed 30000/60000
[INFO] processed 40000/60000
[INFO] processed 50000/60000
[INFO] processed 60000/60000


In [4]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    '''returns a dictionary with keys as client names and values as data shards - (image, label)
        args:
    
                '''
    #  create list of client names
    client_names = ['{}_{}'.format(initial,i) for i in range(num_clients)]
    #  randomize data
    data = list(zip(image_list,label_list))
    random.shuffle(data)
    #  place data at each client
    size = len(data)//num_clients
    shards = [data[i:i+size] for i in range(0, size*num_clients, size)]

    # no of shrads should equa no of clients
    assert(len(shards)==len(client_names))

    return {client_names[i]:shards[i] for i in range(len(client_names))}

In [5]:
#  create clients
clients = create_clients(X_train,y_train,num_clients=10, initial='client')

In [6]:
def batch_data(data_shard, bs=32):
    '''client data shard -> tfds object
        args:
            shard
            batch size
        return
            tfds object'''
    #  sep data and label
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data),list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [7]:
# Process and batch data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#  Processed and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(len(y_test))

2023-02-06 12:01:05.149234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nextg3/miniconda3/envs/fl/lib/python3.9/site-packages/cv2/../../lib64:
2023-02-06 12:01:05.149468: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-06 12:01:05.149485: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DELL-XPS-13-9305): /proc/driver/nvidia/version does not exist
2023-02-06 12:01:05.151128: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild 

In [8]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [9]:
lr = 0.01
coms_round = 100
loss = 'categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate = lr,
                momentum = 0.9
                )

In [19]:
def weight_scaling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    # get bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    # print('bs',bs)
    # Calculater the total data points across the clients
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # print('glocal_count: ',global_count)
    # get total no of data points held by the client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    # print('local_count: ',local_count)
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''Function for scaling a model's weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar*weight[i])
    return weight_final


In [20]:
def  sum_scaled_weights(scaled_weight_list):
    '''Returns the sum of the listed scaled weights.
    scaled average of the weights'''
    avg_grad = list()
    #  get the average grad over all client gradients
    for grad_list_tupel in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tupel, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad


def test_model(X_test, Y_test, model, coms_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    # logits=model.predict(X_test, batch_size=100)
    logits=model.predict(X_test)
    loss = cce(y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis  = 1), tf.argmax(Y_test, axis = 1))
    print('coom_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss


In [23]:
# Initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784,10)

comms_round = 10

# start global training loop
for comm_round in range(comms_round):
    #  get the global model's weights - initial weights for all local models
    global_weights = global_model.get_weights()

    # List to collect local model weights after scaling
    scaled_local_weight_list = list()

    # Randomize client data using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    # Loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784,10)
        local_model.compile(loss=loss,
                            optimizer=optimizer,
                            metrics=metrics)
        # set weights of the global model as weights of the local modelk
        local_model.set_weights(global_weights)

        # Fit local model with client data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        # scale the model weights and add to the list
        # scaling_factor = weight_scaling_factor(clients_batched, client)
        # scaled_weights = scale_model_weights(local_model.get_weights(),scaling_factor)
        # scaled_local_weightsmlp_local = SimpleMLP()
        # local_model = smlp_local.build(784,10)
        # local_model.compile(loss=loss,
        #                     optimizer=optimizer,
        #                     metrics=metrics)
        # # set weights of the global model as weights of the local modelk
        # local_model.set_weights(global_weights)

        # # Fit local model with client data
        # local_model.fit(clients_batched[client], epochs=1, verbose=0)

        # scale the model weights and add to the list
        scaling_factor = weight_scaling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(),scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # Clear session to clean memory
        k.clear_session()
    
    # Get the average overall model
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    # update global model
    global_model.set_weights(average_weights)

    # Test globalmodel
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)


188/188 [==============================] - 0s 630us/step
coom_round: 0 | global_acc: 90.283% | global_loss: 1.6277220249176025
188/188 [==============================] - 0s 619us/step
coom_round: 1 | global_acc: 92.183% | global_loss: 1.5928096771240234
188/188 [==============================] - 0s 608us/step
coom_round: 2 | global_acc: 93.150% | global_loss: 1.5721532106399536
188/188 [==============================] - 0s 612us/step
coom_round: 3 | global_acc: 94.067% | global_loss: 1.5593196153640747
188/188 [==============================] - 0s 748us/step
coom_round: 4 | global_acc: 94.783% | global_loss: 1.5484628677368164
188/188 [==============================] - 0s 614us/step
coom_round: 5 | global_acc: 95.167% | global_loss: 1.5394554138183594
188/188 [==============================] - 0s 629us/step
coom_round: 6 | global_acc: 95.583% | global_loss: 1.5330924987792969
188/188 [==============================] - 0s 625us/step
coom_round: 7 | global_acc: 95.767% | global_loss: 1.5

In [22]:
str(average_weights)

'[<tf.Tensor: shape=(784, 200), dtype=float32, numpy=\narray([[-0.01040198, -0.05578183, -0.05357958, ..., -0.06769065,\n        -0.04030259,  0.04086372],\n       [ 0.06268313, -0.03997761, -0.07568463, ...,  0.00753239,\n         0.02759007,  0.04249723],\n       [-0.00582946,  0.0174942 , -0.02019585, ..., -0.01053266,\n         0.02545312,  0.05473597],\n       ...,\n       [-0.01797263,  0.04460895,  0.02997313, ..., -0.0529381 ,\n         0.0279701 , -0.06740224],\n       [ 0.0193205 , -0.01978557, -0.0189405 , ..., -0.00771479,\n        -0.02440459, -0.00386408],\n       [ 0.06105473,  0.06315143,  0.01426234, ...,  0.03281127,\n         0.04488137, -0.06221747]], dtype=float32)>, <tf.Tensor: shape=(200,), dtype=float32, numpy=\narray([ 0.02121967,  0.01608862,  0.00875596,  0.01596739,  0.00928205,\n       -0.01048895,  0.0279254 , -0.02941848, -0.02364868,  0.00264643,\n       -0.01464437,  0.00828531,  0.00178428,  0.00848354,  0.03125082,\n        0.10475545,  0.094607  ,  0

### Export architecture to json file

In [ ]:

json_string = global_model.to_json()

In [ ]:
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 784], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "batch_input_shape": [null, 784], "units": 200, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Activation", "config": {"name": "activation", "trainable": true, "dtype": "float32", "activation": "relu"}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 200, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "Gl

### Export weights as h5 file

In [ ]:
global_model.save_weights('weights.h5')

### Export weights as a list

In [ ]:
import pickle

with open('weights.pkl', 'wb') as f:
    pickle.dump(average_weights, f)